In [1]:
from langchain.vectorstores import DeepLake
import os
import json 
from dotenv import load_dotenv
load_dotenv('./.env')
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader
from langchain.document_loaders import UnstructuredPowerPointLoader
from langchain.document_loaders import CSVLoader


/home/elshan/Projects/LunaAI/env_q_n_a/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.4) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ACTIVELOOP_TOKEN = os.getenv('ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID = os.getenv('ACTIVELOOP_ORG_ID')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
os.environ["ACTIVELOOP_ORG_ID"] = ACTIVELOOP_ORG_ID

In [3]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

## **Creation of the Deep Lake dataset**

In [4]:
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVELOOP_ORG_ID
# my_activeloop_dataset_name = "QA_knowledgebase"
my_activeloop_dataset_name = "QA_knowledgebase_v2"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Deep Lake Dataset in hub://echalabiyev/QA_knowledgebase_v2 already exists, loading from the storage


In [5]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

## **PyPDFLoader (PDF)**

!pip install -q pypdf

In [7]:

def convert_and_split(text_splitter, file_path: str) -> list:
    '''
    It converts the file to a textual representation, adds metadata, and
    calls the text_splitter to split long sequences.
    It returns a list of Documents or None is there is any error in the 
    file conversion.
    '''


    texts = None
    try:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            pages = loader.load_and_split()
            texts = text_splitter.split_documents(pages)

        elif file_path.endswith('.docx') or file_path.endswith('.doc'):
            loader = UnstructuredWordDocumentLoader(file_path)
            docs = loader.load_and_split()
            texts = text_splitter.split_documents(docs)
        
        elif file_path.endswith('.txt'):
            loader = TextLoader(file_path)
            docs = loader.load_and_split()
            texts = text_splitter.split_documents(docs)

        elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            loader = UnstructuredExcelLoader(file_path)
            docs = loader.load_and_split()
            texts = text_splitter.split_documents(docs)

        elif file_path.endswith('.csv'):
            loader = CSVLoader(file_path)
            docs = loader.load_and_split()
            texts = text_splitter.split_documents(docs)
        
        elif file_path.endswith('.pptx') or file_path.endswith('.ppt'):
            loader = UnstructuredPowerPointLoader(file_path)
            docs = loader.load_and_split()
            texts = text_splitter.split_documents(docs)

        else:
            print(f"Error: invalid file type: {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

    return texts

In [8]:
file_path ="/home/elshan/Projects/LunaAI/pdf_data/Regolamento Aziendale.pdf"

## **Recursive Character Text Splitter**

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# docs = text_splitter.split_documents(pages)
for i in convert_and_split(text_splitter, file_path):
    print(i.page_content)
    print("#"*50)

Luna Labs Srl 
Via Parini 9 • 20121 Milano • Codice Fiscale/Partita Iva: 10220960966  
telefono : +39 02 37.90.20.29 • e-mail : info@lunalabs.it  • sito web: www.lunalabs.it  
 
 
Regolamento Luna   
 
Data emissione:  01/01/2021  
 
Il presente documento ha lo scopo di:  
 
• Rendere note le semplici regole ritenute essenziali per regolamentare il rapporto tra  
ciascun lavoratore e l'azienda.  
 
• Definire principi comportamentali in linea con la volontà della Direzione aziendale.  
 
 REGOLAMENTAZIONE RAPPORTO CON L'AZIENDA  
 
Casella di posta personale  
 
Tutte le comunicazioni rivolte dall’azienda avverranno attraverso la casel la di posta  
personale.  
 
Time sheet  
 
La compilazione del time -sheet sul portale  lunalabs.altamiraweb.com  (vedi le Istruzioni)  
Giornalmente devono sempre essere rendicontate 8 ore (o tutte svolte presso il  
Cliente o diversamente imputando la relativa causale (ferie, permessi…) La pausa  
pranzo non deve essere indicata.
####################

In [39]:
# for i in docs:
#     print(len(i.page_content))

In [56]:
# all_texts, all_metadatas = [], []
# for d in convert_and_split(text_splitter, file_path):
#     # print(d)
#     all_texts.append(d.page_content)
#     all_metadatas.append(d.metadata)

# all_metadatas
# all_texts

In [55]:
# db.add_documents(documents=docs)

## **Character Text Splitter**

In [13]:
from langchain.text_splitter import CharacterTextSplitter


In [14]:
text_splitter = CharacterTextSplitter(separator='\n',chunk_size=1000, chunk_overlap=150, length_function = len,)
# docs = convert_and_split(text_splitter, file_path)
# docs
for i in convert_and_split(text_splitter, file_path):
    print(i.page_content)
    print("#"*50)


Luna Labs Srl 
Via Parini 9 • 20121 Milano • Codice Fiscale/Partita Iva: 10220960966  
telefono : +39 02 37.90.20.29 • e-mail : info@lunalabs.it  • sito web: www.lunalabs.it  
 
 
Regolamento Luna   
 
Data emissione:  01/01/2021  
 
Il presente documento ha lo scopo di:  
 
• Rendere note le semplici regole ritenute essenziali per regolamentare il rapporto tra  
ciascun lavoratore e l'azienda.  
 
• Definire principi comportamentali in linea con la volontà della Direzione aziendale.  
 
 REGOLAMENTAZIONE RAPPORTO CON L'AZIENDA  
 
Casella di posta personale  
 
Tutte le comunicazioni rivolte dall’azienda avverranno attraverso la casel la di posta  
personale.  
 
Time sheet  
 
La compilazione del time -sheet sul portale  lunalabs.altamiraweb.com  (vedi le Istruzioni)  
Giornalmente devono sempre essere rendicontate 8 ore (o tutte svolte presso il  
Cliente o diversamente imputando la relativa causale (ferie, permessi…) La pausa  
pranzo non deve essere indicata.
####################

In [126]:
pretty_print_docs(docs)

Document 1:

Luna Labs Srl 
Via Parini 9 • 20121 Milano • Codice Fiscale/Partita Iva: 10220960966 
telefono: +39 02 37.90.20.29 • e-mail: info@lunalabs.it • sito web: www.lunalabs.it  
 
 
Regolamento Luna  
 
Data emissione: 01/01/2021 
 
Il presente documento ha lo scopo di: 
 
• Rendere note le semplici regole ritenute essenziali per regolamentare il rapporto tra 
ciascun lavoratore e l'azienda. 
 
• Definire principi comportamentali in linea con la volontà della Direzione aziendale. 
 
 
REGOLAMENTAZIONE RAPPORTO CON L'AZIENDA 
 
Casella di posta personale 
 
Tutte le comunicazioni rivolte dall’azienda avverranno attraverso la casella di posta 
personale. 
 
Time sheet 
 
La compilazione del time-sheet sul portale lunalabs.altamiraweb.com (vedi le Istruzioni) 
Giornalmente devono sempre essere rendicontate 8 ore (o tutte svolte presso il 
Cliente o diversamente imputando la relativa causale (ferie, permessi…) La pausa 
pranzo non deve essere indicata.
-----------------------------

In [120]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = CharacterTextSplitter(separator='\n',chunk_size=1000, chunk_overlap=150, length_function = len,)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100,separators=["\n\n", "\n", " ", ""])

# loader = UnstructuredWordDocumentLoader(file_path)
# d = loader.load()
texts = text_splitter.split_documents(docs)
texts

[Document(page_content="Luna Labs Srl \nVia Parini 9 • 20121 Milano • Codice Fiscale/Partita Iva: 10220960966 \ntelefono: +39 02 37.90.20.29 • e-mail: info@lunalabs.it • sito web: www.lunalabs.it  \n \n \nRegolamento Luna  \n \nData emissione: 01/01/2021 \n \nIl presente documento ha lo scopo di: \n \n• Rendere note le semplici regole ritenute essenziali per regolamentare il rapporto tra \nciascun lavoratore e l'azienda. \n \n• Definire principi comportamentali in linea con la volontà della Direzione aziendale. \n \n\uf0b7 \nREGOLAMENTAZIONE RAPPORTO CON L'AZIENDA \n \nCasella di posta personale \n \nTutte le comunicazioni rivolte dall’azienda avverranno attraverso la casella di posta \npersonale. \n \nTime sheet \n \nLa compilazione del time-sheet sul portale lunalabs.altamiraweb.com (vedi le Istruzioni) \nGiornalmente devono sempre essere rendicontate 8 ore (o tutte svolte presso il \nCliente o diversamente imputando la relativa causale (ferie, permessi…) La pausa \npranzo non deve e

In [121]:
pretty_print_docs(texts)

Document 1:

Luna Labs Srl 
Via Parini 9 • 20121 Milano • Codice Fiscale/Partita Iva: 10220960966 
telefono: +39 02 37.90.20.29 • e-mail: info@lunalabs.it • sito web: www.lunalabs.it  
 
 
Regolamento Luna  
 
Data emissione: 01/01/2021 
 
Il presente documento ha lo scopo di: 
 
• Rendere note le semplici regole ritenute essenziali per regolamentare il rapporto tra 
ciascun lavoratore e l'azienda. 
 
• Definire principi comportamentali in linea con la volontà della Direzione aziendale. 
 
 
REGOLAMENTAZIONE RAPPORTO CON L'AZIENDA 
 
Casella di posta personale 
 
Tutte le comunicazioni rivolte dall’azienda avverranno attraverso la casella di posta 
personale. 
 
Time sheet 
 
La compilazione del time-sheet sul portale lunalabs.altamiraweb.com (vedi le Istruzioni) 
Giornalmente devono sempre essere rendicontate 8 ore (o tutte svolte presso il 
Cliente o diversamente imputando la relativa causale (ferie, permessi…) La pausa 
pranzo non deve essere indicata.
-----------------------------

In [ ]:
dt = {}
for i in texts:
    dt[len(i.page_content)] = i.page_content
    print(len(i.page_content))
    print(i.page_content)

In [27]:
# loader = PyPDFLoader("/home/elshan/Projects/LunaAI/pdf_data/Regolamento Aziendale.pdf")
# pages = loader.load_and_split()

# text_splitter = CharacterTextSplitter(separator='\n',chunk_size=1000, chunk_overlap=150, length_function = len,)
# docs = text_splitter.split_documents(pages)
# for i in docs:
#     print(len(i.page_content))
#     print(i.page_content)
# db.add_documents(documents=docs)


## 1. Using **PromtTemplate** to extract data from Vecot db

In [92]:
from langchain import PromptTemplate
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

In [80]:
query = "raccontami riguardo ai possibili stati della postazione in Luna Labs"
docs = db.similarity_search(query, k=2)
docs

[Document(page_content="Luna Labs Srl \nVia Parini 9 • 20121 Milano • Codice Fiscale/Partita Iva: 10220960966  \ntelefono : +39 02 37.90.20.29 • e-mail : info@lunalabs.it  • sito web: www.lunalabs.it  \n \n \nRegolamento Luna   \n \nData emissione:  01/01/2021  \n \nIl presente documento ha lo scopo di:  \n \n• Rendere note le semplici regole ritenute essenziali per regolamentare il rapporto tra  \nciascun lavoratore e l'azienda.  \n \n• Definire principi comportamentali in linea con la volontà della Direzione aziendale.  \n \n\uf0b7 REGOLAMENTAZIONE RAPPORTO CON L'AZIENDA  \n \nCasella di posta personale  \n \nTutte le comunicazioni rivolte dall’azienda avverranno attraverso la casel la di posta  \npersonale.  \n \nTime sheet  \n \nLa compilazione del time -sheet sul portale  lunalabs.altamiraweb.com  (vedi le Istruzioni)  \nGiornalmente devono sempre essere rendicontate 8 ore (o tutte svolte presso il  \nCliente o diversamente imputando la relativa causale (ferie, permessi…) La pausa

In [72]:
print(docs[0].page_content)

se presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati
il box colorato indica che è occupata da un membro del team corrispondente a quel colore
il box grigio indica che la postazione è occupata da più membri di diversi team
Scegli una postazione libera e premi “Conferma” per prenotarla.
Ora hai un posto dove sederti in ufficio.
Prenotazione sala riunioni
Hai bisogno di prenotare una sala riunioni? Segui la stessa procedura selezionando la mappa dal menu in basso o il bottone “Prenota postazioni” dall’home page. Scegli i giorni, imposta l'ora di inizio e fine incontro e conferma.
Potrai modificare o eliminare le tue prenotazioni selezionando la voce del menu “Prenotazioni”.
I team
Ad ogni team corrisponde un colore diverso. Selezionando nel menu la voce “Team” potrai scoprire i team presenti in azienda e vedere il colore e il nome associati ad ognuno di loro. Cerca il tuo team in base al ruolo che hai in azienda. 
Feedback


In [93]:
# retrieve relevant chunks
query = "quale la legenda dei diversi stati delle postazioni?"
docs = db.similarity_search(query, k=2)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)
# print( prompt_formatted)

In [94]:
from langchain import OpenAI
# generate answer
llm = OpenAI(model="text-davinci-003", temperature=0)
answer = llm(prompt_formatted)
print(answer)

 Il box bianco indica che la sedia è libera. Se presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati. Il box colorato indica che è occupata da un membro del team corrispondente a quel colore. Il box grigio indica che la postazione è occupata da più membri di diversi team.


## 2. Extracting data from vector db without the Prompt

In [56]:
# we create a RetrievalQAWithSourcesChain chain, which is very similar to a
# standard retrieval QA chain but it also keeps track of the sources of the
# retrieved documents

from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    chain_type="stuff",
                                                    retriever=db.as_retriever())

In [57]:
# We generate a response to a query using the chain. The response object is a dictionary containing
# an "answer" field with the textual answer to the query, and a "sources" field containing a string made
# of the concatenation of the metadata["source"] strings of the retrieved documents.
d_response = chain({"question": "quale la legenda dei diversi stati delle postazioni?"})

print("Response:")
print(d_response["answer"])
print("Sources:")
for source in d_response["sources"].split(", "):
    print("- " + source)

Response:
 Il box bianco indica che la sedia è libera, il box colorato indica che è occupata da un membro del team corrispondente a quel colore, il box grigio indica che la postazione è occupata da più membri di diversi team.

Sources:
- /home/elshan/Projects/LunaAI/pdf_data/LunaPlaces_testo.docx


## 3. Retriving the data with **ContextualCompressionRetriever**

In [87]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# create GPT3 wrapper
llm = OpenAI(model="text-davinci-003", temperature=0)

# create compressor for the retriever
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
	base_compressor=compressor,
	base_retriever=db.as_retriever()
)

In [88]:
# retrieving compressed documents
retrieved_docs = compression_retriever.get_relevant_documents(
	"quale la legenda dei diversi stati delle postazioni?"
)
# print(retrieved_docs[0].page_content)

In [89]:
retrieved_docs

[Document(page_content='se presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati\nil box colorato indica che è occupata da un membro del team corrispondente a quel colore\nil box grigio indica che la postazione è occupata da più membri di diversi team\nI team\nAd ogni team corrisponde un colore diverso.', metadata={'source': '/home/elshan/Projects/LunaAI/pdf_data/LunaPlaces_testo.docx'}),
 Document(page_content='Premendo la “i” di info in basso a destra puoi scoprire la legenda dei diversi stati delle postazioni:\nil box bianco indica che la sedia è libera\nse presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati', metadata={'source': '/home/elshan/Projects/LunaAI/pdf_data/LunaPlaces_testo.docx'})]

In [97]:


pretty_print_docs(retrieved_docs)

Document 1:

se presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati
il box colorato indica che è occupata da un membro del team corrispondente a quel colore
il box grigio indica che la postazione è occupata da più membri di diversi team
I team
Ad ogni team corrisponde un colore diverso.
----------------------------------------------------------------------------------------------------
Document 2:

Premendo la “i” di info in basso a destra puoi scoprire la legenda dei diversi stati delle postazioni:
il box bianco indica che la sedia è libera
se presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati


In [90]:
retrieved_chunks = [doc.page_content for doc in retrieved_docs]
retrieved_chunks

['se presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati\nil box colorato indica che è occupata da un membro del team corrispondente a quel colore\nil box grigio indica che la postazione è occupata da più membri di diversi team\nI team\nAd ogni team corrisponde un colore diverso.',
 'Premendo la “i” di info in basso a destra puoi scoprire la legenda dei diversi stati delle postazioni:\nil box bianco indica che la sedia è libera\nse presente anche il pallino arancione significa che la postazione è disponibile sono in alcuni dei giorni selezionati']

In [91]:
# retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="text-davinci-003", temperature=0)
answer = llm(prompt_formatted)
print(answer)

 Le postazioni in Luna Labs possono avere tre stati diversi. Il box bianco indica che la sedia è libera, il box colorato indica che è occupata da un membro del team corrispondente a quel colore, e il box grigio indica che la postazione è occupata da più membri di diversi team. Inoltre, se presente anche il pallino arancione significa che la postazione è disponibile solo in alcuni dei giorni selezionati.
